# Part 1

### ECF Power Flow

In order to understand ECF power flow analysis (or any other formulation), we first need to establish what equations the model uses to represent the physics of the electrical network and how those equations are then converted into a linear system in order to calculate the required information. The basic elements of the network include transmission lines, transformers, shunts, and simplified forms of generation, load, and a representation of 'slack' in the system, and power flow analysis must have a way to represent these elements. Each element is attached to one or two buses on the network, and enforces different constraints on the bus(es) it is connected to. For instance, a generator is described as a PV bus that supplies a fixed amount of real power and sets a voltage magnitude at the bus that it is attached to. Loads are represented as PQ buses that consume a fixed amount of real and reactive power. Finally, a slack bus is a reference point for the rest of the system that acts as a peusdo-infinite generator, attempting to set a fixed voltage magnitude and angle at its bus (and acts as a representation of power supply mismatch). 

In classic PQV formulation, we attempt to balance the real power ($P$) and reactive power ($Q$) at each bus in the system. Depending on the type of bus, we have a different set of 2 known and 2 unknown values: $P$, $Q$, $V$ (the voltage magnitude at the bus), and $\theta$ (the voltage angle at the bus). These unknowns are what the PQV solves for with its linearized system, with each circuit element described using some combination of these 4 values in a set of equations, primarily summations of $P$ and $Q$ [1].

With ECF power flow analysis, rather than attempting to balance $P$ and $Q$ directly, we instead model the electric grid as a circuit and utilize the same kirchhoff's current law (KCL) and kirchhoff's voltage law (KVL) equations that are utilized in time-domain analysis for circuitry. Using equations that govern the power/voltage/current relationship for different bus types allows us to define the set of non-linear equations that we want to linearize and solve for using Newton-Raphson (NR). For PV and PQ nodes, a main equation is $P + jQ = (V_R + JV_I)(I_R - jI_I)$, or expressed for currents as $I_R + jI_I = \frac{P - jQ}{V_R + jV_I}$ for use with KCL. Crucially, these equations include real and complex terms and are not directly differentiable. In order to make the system solvable, the circuit is divided into real and imaginary components where we solve for $V_R$ and $V_I$ as separate unknowns. While these split equations are still non-linear, they are at least differentiable and can be embedded into the linear system. The final split current equations are reformulated as:
$$I_R = \frac{PV_R + QV_I}{V_R^2 + V_I^2}$$
$$I_I = \frac{PV_I - QV_R}{V_R^2 + V_I^2}$$

Note that the signs of the $P$ and $Q$ terms are inverted for PV buses that injecting current rather than consuming it. In addition, PV buses assume a fixed voltage magnitude rather than a set Q value. Since Q is a new unknown and we need an equation describing the voltage magnitude constraint, we need an extra equation before it is possible to solve the system of equations: $V_R^2 + V_I^2 = V_{Set}^2$. These equations are linearized using taylor expansion and are then solved for with NR [2]. 

Given that an iteration of NR can be expressed as $\nabla f(x_k)x_{k+1} = -f(x) + \nabla f(x_k)x_k$, the fully linearized equations for PQ bus are:

##### Real:

$$\frac{\partial I_{R(k)}}{\partial V_{R(k)}} = \frac{P( V_{I(k)}^2 - V_{R(k)}^2 ) - 2QV_{R(k)}V_{I(k)}}{(V_{I(k)}^2 + V_{R(k)}^2)^2}$$

$$\frac{\partial I_{R(k)}}{\partial V_{I(k)}} = \frac{Q( V_{R(k)}^2 - V_{I(k)}^2 ) - 2PV_{R(k)}V_{I(k)}}{(V_{I(k)}^2 + V_{R(k)}^2)^2}$$

$$I_{RL(k+1)} = \frac{\partial I_{R(k)}}{\partial V_{R(k)}}(V_{R(k+1)}) + \frac{\partial I_{R(k)}}{\partial V_{I(k)}}(V_{I(k+1)}) - I_{R(k)} - \frac{\partial I_{R(k)}}{\partial V_{R(k)}}(V_{R(k)}) - \frac{\partial I_{R(k)}}{\partial V_{I(k)}}(V_{I(k)})$$

##### Imaginary:

$$\frac{\partial I_{I(k)}}{\partial V_{R(k)}} = \frac{P( V_{I(k)}^2 - V_{R(k)}^2 ) + 2QV_{R(k)}V_{I(k)}}{(V_{I(k)}^2 + V_{R(k)}^2)^2}$$

$$\frac{\partial I_{I(k)}}{\partial V_{I(k)}} = \frac{Q( V_{I(k)}^2 - V_{R(k)}^2 ) - 2PV_{R(k)}V_{I(k)}}{(V_{I(k)}^2 + V_{R(k)}^2)^2}$$

$$I_{I(k+1)} = \frac{\partial I_{I(k)}}{\partial V_{R(k)}}(V_{R(k+1)}) + \frac{\partial I_{I(k)}}{\partial V_{I(k)}}(V_{I(k+1)}) - I_{I(k)} - \frac{\partial I_{I(k)}}{\partial V_{R(k)}}(V_{R(k)}) - \frac{\partial I_{I(k)}}{\partial V_{I(k)}}(V_{I(k)})$$

The derivations for the PV bus contributions to current are ommitted for brevity, but are very similar to their PQ counterparts. The $V_{Set}$ equation must also be linearized and given a form for Newton-Raphson:

$$2V_{R(k+1)} + 2V_{I(k+1)} = 2V_{R(k)} + 2V_{I(k)} + V_{Set}^2$$

These equations are combined with equations that govern linear devices: branches, transformers, and shunts to create the full linear system that we then iteratively solve for.

### Implementation Discussion

My implementation closely follows the supplied structure of the original codebase. The primary simulator is contained in the PowerFlow class, with independent models for each element of the power system. The only worthwhile note is that matrix construction is centralized in the MatrixBuilder class, which handles both sparse and dense construction. Test cases can be executed using `run_solver.py`.

### Results & Computational Performance

The solution and computational performance for the GS-4 and IEEE-14 cases are evalutated below and compared to results from Matpower. Unfortunately, the IEEE-118 and ACTIVSg500 cases do not converge with the simulator and are omitted.

**GS-4 Test Case (simulator):**

In [1]:
from lib.Solve import solve
from parsers.parser import parse_raw
from lib.settings import Settings

raw_data = parse_raw("testcases/GS-4_prior_solution.RAW")
settings = Settings(max_iters=30, limiting=False, use_sparse=True)
result = solve(raw_data, settings)

result.display()

Time to parse the file is: 0.00174
Running power flow solver...
Power flow solver converged after 5 iterations.
Ran for 0.005 seconds
Bus Results:
Bus 1 V_mag (pu): 1.000, V_ang (deg): -0.000
Bus 2 V_mag (pu): 0.982, V_ang (deg): -0.976
Bus 3 V_mag (pu): 0.969, V_ang (deg): -1.872
Bus 4 V_mag (pu): 1.020, V_ang (deg): 1.523
Generator Results:
Generator @ bus 4 P (MW): 318.00, Q (MVar): 181.43
Slack @ bus 4 P (MW): -186.81, Q (MVar): -0.00


**GS-4 Test Case (Matpower):**

Note that Matpower converged in 3 iterations.

In [2]:
from scipy.io import loadmat
from lib.process_results import display_mat_comparison

mat = loadmat("testcases/output-GS-4.mat")
display_mat_comparison(mat, result)

Bus: 1 V_mag diff: 0.0000 V_ang diff: -0.0000
Bus: 2 V_mag diff: 0.0000 V_ang diff: -0.0001
Bus: 3 V_mag diff: 0.0000 V_ang diff: -0.0001
Bus: 4 V_mag diff: 0.0000 V_ang diff: 0.0001


**IEEE-14 Test Case (simulator):**

In [3]:
casename = 'testcases/IEEE-14_prior_solution.RAW'
raw_data = parse_raw(casename)
settings = Settings(debug=False, max_iters=30, limiting=False, use_sparse=True)
result = solve(raw_data, settings)
result.display()

Time to parse the file is: 0.00250
Running power flow solver...
Power flow solver converged after 22 iterations.
Ran for 0.022 seconds
Bus Results:
Bus 1 V_mag (pu): 1.060, V_ang (deg): -0.000
Bus 2 V_mag (pu): 1.045, V_ang (deg): -4.983
Bus 3 V_mag (pu): 1.010, V_ang (deg): -12.724
Bus 4 V_mag (pu): 1.018, V_ang (deg): -10.313
Bus 5 V_mag (pu): 1.020, V_ang (deg): -8.774
Bus 6 V_mag (pu): 1.070, V_ang (deg): -14.218
Bus 7 V_mag (pu): 1.062, V_ang (deg): -13.358
Bus 8 V_mag (pu): 1.090, V_ang (deg): -13.358
Bus 9 V_mag (pu): 1.056, V_ang (deg): -14.935
Bus 10 V_mag (pu): 1.051, V_ang (deg): -15.093
Bus 11 V_mag (pu): 1.057, V_ang (deg): -14.787
Bus 12 V_mag (pu): 1.055, V_ang (deg): -15.072
Bus 13 V_mag (pu): 1.050, V_ang (deg): -15.152
Bus 14 V_mag (pu): 1.036, V_ang (deg): -16.028
Generator Results:
Generator @ bus 2 P (MW): 40.00, Q (MVar): 43.56
Generator @ bus 3 P (MW): 0.00, Q (MVar): 25.08
Generator @ bus 6 P (MW): 0.00, Q (MVar): 12.73
Generator @ bus 8 P (MW): 0.00, Q (MVar): 

**IEEE-14 Test Case (Matpower):**

Note that Matpower converged in 1 iteration.

In [4]:
mat = loadmat("testcases/output-IEEE-14.mat")
display_mat_comparison(mat, result)

Bus: 1 V_mag diff: -0.0000 V_ang diff: -0.0000
Bus: 2 V_mag diff: 0.0000 V_ang diff: -0.0003
Bus: 3 V_mag diff: 0.0000 V_ang diff: 0.0012
Bus: 4 V_mag diff: 0.0000 V_ang diff: -0.0000
Bus: 5 V_mag diff: 0.0000 V_ang diff: -0.0003
Bus: 6 V_mag diff: 0.0000 V_ang diff: 0.0026
Bus: 7 V_mag diff: 0.0000 V_ang diff: 0.0017
Bus: 8 V_mag diff: 0.0000 V_ang diff: 0.0017
Bus: 9 V_mag diff: 0.0000 V_ang diff: 0.0036
Bus: 10 V_mag diff: 0.0000 V_ang diff: 0.0039
Bus: 11 V_mag diff: 0.0000 V_ang diff: 0.0034
Bus: 12 V_mag diff: 0.0000 V_ang diff: 0.0038
Bus: 13 V_mag diff: 0.0000 V_ang diff: 0.0040
Bus: 14 V_mag diff: 0.0000 V_ang diff: 0.0055


**Computational Performance:**

The relative compute time for the 14 bus case is displayed in the output below. From empirical testing, the sparse matrix construction yields a minor improvement in compute time. Ideally larger test cases would reveal a more definitive result.

In [7]:
casename = 'testcases/IEEE-14_prior_solution.RAW'
raw_data = parse_raw(casename)

settings = Settings(debug=False, max_iters=30, limiting=False, use_sparse=False)
result = solve(raw_data, settings)

settings = Settings(debug=False, max_iters=30, limiting=False, use_sparse=True)
result = solve(raw_data, settings)

Time to parse the file is: 0.00208
Running power flow solver...
Power flow solver converged after 22 iterations.
Ran for 0.074 seconds
Running power flow solver...
Power flow solver converged after 22 iterations.
Ran for 0.035 seconds


#### Citations

[1] G. Andersson, “Power System Analysis.” ITET ETH Zurich, Sep. 2012.
[2] A. Pandey, “Robust Steady-State Analysis of Power Grid using Equivalent Circuit Formulation  with Circuit Simulation Methods.” Carnegie Mellon University, Dec. 2018.

